<a href="https://colab.research.google.com/github/tukamilano/anagram/blob/main/%E3%81%B2%E3%82%89%E3%81%8C%E3%81%AABert%E3%81%AE%E5%AD%A6%E7%BF%92.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install fugashi unidic-lite datasets tqdm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.4/47.4 MB 17.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 694.9/694.9 kB 36.7 MB/s eta 0:00:00
  Created wheel for unidic-lite: filename=unidic_lite-1.0.8-py3-none-any.whl size=47658817 sha256=89cd1732fe10d7910bc38f98958afb3097fdf0c3ffaffea3eae78a9206713985
  Stored in directory: /root/.cache/pip/wheels/5e/1f/0f/4d43887e5476d956fae828ee9b6687becd5544d68b51ed633d
Successfully built unidic-lite


In [11]:
import shutil
import os
from google.colab import drive

drive.mount('/content/drive')
file_name = "aozora_hiragana.txt" #あとはサンプルも?

# Google Driveのファイルパスを指定
file_path = f"/content/drive/MyDrive/anagram/{file_name}"

# コピー先のファイル名を指定
destination_file_name = file_name

# Google Driveがマウントされているか確認し、ファイルをコピー
if os.path.exists(file_path):
    shutil.copy(file_path, destination_file_name)
    print(f"'{file_path}' を '{destination_file_name}' にコピーしました。")
else:
    print(f"エラー: ファイル '{file_path}' が見つかりません。Google Driveがマウントされているか、パスが正しいか確認してください。")
    print("Google Driveをマウントするには、次のコードを実行してください: from google.colab import drive; drive.mount('/content/drive')")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
'/content/drive/MyDrive/anagram/aozora_hiragana.txt' を 'aozora_hiragana.txt' にコピーしました。


In [3]:
from transformers import AutoTokenizer

# 日本語BERT（東北大）の tokenizer
tokenizer = AutoTokenizer.from_pretrained(
    "cl-tohoku/bert-base-japanese-v3"
)

def tokenize_morphemes(text: str, tokenizer):
    """
    text: スペース区切り形態素列の文字列
    例: "その よくぼう を みたす こと も おさえる こと も でき なかっ た の に"

    戻り値:
        tokens: subword レベルの token のリスト
        token_ids: それを vocab ID に変換したもの
    """

    morphemes = text.strip().split()   # ← 形態素を事前に分割済み

    tokens = []
    for m in morphemes:
        # 形態素1つずつ tokenizer に渡して token 化
        sub_tokens = tokenizer.tokenize(m)
        tokens.extend(sub_tokens)

    # token → vocab ID に変換
    token_ids = tokenizer.convert_tokens_to_ids(tokens)
    return tokens, token_ids


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/251 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

In [4]:
text = "その よくぼう を みたす こと も おさえる こと も でき なかっ た の に"

tokens, token_ids = tokenize_morphemes(text, tokenizer)

print(tokens)
print(token_ids)

['その', 'よく', 'ぼう', 'を', 'み', '##た', '##す', 'こと', 'も', 'おさ', '##える', 'こと', 'も', 'でき', 'なかっ', 'た', 'の', 'に']
[12491, 13304, 22145, 500, 481, 7433, 7243, 12489, 484, 22148, 12580, 12489, 484, 12535, 12581, 449, 464, 461]


In [5]:
!pip install transformers datasets accelerate sentencepiece

In [29]:
!head -n 10000 aozora_hiragana.txt > aozora_hiragana_10000.txt

In [31]:
# continual_pretrain_bert_japanese_v3_colab.py

import os
from datasets import load_dataset
from transformers import (
    AutoTokenizer,
    AutoModelForMaskedLM,
    DataCollatorForWholeWordMask,
    DataCollatorForLanguageModeling,
    Trainer,
    TrainingArguments,
)

import wandb

wandb.init(project="bert_continual_pretrain")

MODEL_NAME = "tohoku-nlp/bert-base-japanese-v3"

# 1. tokenizer & model ロード
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
model = AutoModelForMaskedLM.from_pretrained(MODEL_NAME)


# 2. 形態素単位 tokenization
def tokenize_morpheme_sentence(examples, max_length=128):
    # examples["text"] はリスト
    texts = examples["text"]

    # 各文を形態素分割し、リストのリストを作る
    batch_morphemes = [t.strip().split() for t in texts]

    # tokenizer にリストのリストを渡す（これが正しい）
    encoded = tokenizer(
        batch_morphemes,
        is_split_into_words=True,
        max_length=max_length,
        truncation=True,
        padding="max_length",
    )
    return encoded


def main():

    # 3. データ読み込み（1行1文）
    data_files = {
        "train": "aozora_hiragana_10000.txt",
        "validation": "aozora_sample.txt",
    }

    raw_datasets = load_dataset("text", data_files=data_files)

    # 4. Tokenize
    tokenized = raw_datasets.map(
        tokenize_morpheme_sentence,
        batched=True,
        batch_size=500,
        remove_columns=["text"],
        num_proc=2,         # Colab CPU を使って高速化
        desc="Tokenizing"
    )

    # 5. MLM masking
    data_collator = DataCollatorForLanguageModeling( #DataCollatorForWholeWordMaskを使ってみたい
        tokenizer=tokenizer,
        mlm=True,
        mlm_probability=0.15,
    )

    # 6. TrainingArguments
    training_args = TrainingArguments(
        output_dir="./bert_japanese_v3_continual",
        overwrite_output_dir=True,
        num_train_epochs=1,                     # ★ epoch = 1
        per_device_train_batch_size=16,         # Colab GPU 安全設定
        per_device_eval_batch_size=16,
        learning_rate=3e-5,
        weight_decay=0.01,
        warmup_steps=500,
        logging_strategy="steps",
        logging_steps=20,
        save_steps=2000,
        eval_steps=2000,
        report_to=["wandb"],
        save_total_limit=1,                     # 保存少なくして軽量化
        fp16=True,                              # Colab GPU (T4) なら True 推奨
    )

    # 7. Trainer
    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=tokenized["train"],
        eval_dataset=tokenized["validation"],
        data_collator=data_collator,
    )

    # 8. 学習
    trainer.train()

    # 9. 最終モデル保存（tokenizerは保存しない）
    trainer.save_model("./bert_japanese_v3_continual/final")


if __name__ == "__main__":
    main()



train/epoch,▁
train/global_step,▁
total_flos,2961131765760.0
train/epoch,1
train/global_step,3
train_loss,4.49709
train_runtime,15.3766
train_samples_per_second,2.927
train_steps_per_second,0.195


Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Tokenizing (num_proc=2):   0%|          | 0/10000 [00:00<?, ? examples/s]

Tokenizing (num_proc=2):   0%|          | 0/45 [00:00<?, ? examples/s]

Step,Training Loss
20,4.167200
40,3.921100
60,3.824500
80,3.670500
100,3.655000
120,3.566400
140,3.542000
160,3.567900
180,3.381100
200,3.273600
